In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time

driver = webdriver.Chrome()
    
def open_search_page(page):
    url_used = "https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page=" + str(page) + "&orderBy=relevance"
    #url_used = self.url + str(page) + self.url_end
    #print(self.url_used)
    driver.get(url_used)
    #print(driver.title)
    assert "Immoweb" in driver.title
    time.sleep(3)
    cookie_button = driver.find_element(By.XPATH, "//*[@id='uc-btn-accept-banner']")
    time.sleep(1)
    cookie_button.click()
    time.sleep(1)
    house_urls = []
    house_tag = driver.find_elements(By.XPATH, "//h2/a[@href]")
    for house_url in house_tag:
        if "From" not in house_url.get_attribute('aria-label'):
            house_urls.append(house_url.get_attribute('href'))
    for x in house_urls:
        driver.get(x)
        page_contain = {}
        table_rows = driver.find_elements(By.XPATH, "//tr[@class = 'classified-table__row']")
        for table_row in table_rows:
            try:
                tr_header = table_row.find_element(By.XPATH, ".//th").text
                tr_data = ''.join(table_row.find_element(By.XPATH, ".//td").text)
                
            except NoSuchElementException:
                pass
            page_contain[tr_header] = tr_data
        print(page_contain)

#automatize scrapping data
def new_search_page_scraping():
    for page in range(1,5):
        open_search_page(page)
        continue

            #new_search_page()
            #if captcha --> action    

new_search_page_scraping()

{'Available as of': 'After signing the deed', 'Construction year': '1989', 'Building condition': 'As new', 'Street frontage width': '12 m', 'Number of frontages': '4', 'Covered parking spaces': '2', 'Outdoor parking spaces': '2', 'Surroundings type': 'Landscape', 'Living area': '404 m²\nsquare meters', 'Living room surface': '55 m²\nsquare meters', 'Dining room': 'Yes', 'Kitchen type': 'Hyper equipped', 'Kitchen surface': '16 m²\nsquare meters', 'Bedrooms': '4', 'Bedroom 1 surface': '12 m²\nsquare meters', 'Bedroom 2 surface': '16 m²\nsquare meters', 'Bedroom 3 surface': '27 m²\nsquare meters', 'Dressing room': 'No', 'Bathrooms': '2', 'Toilets': '2', 'Laundry room': 'Yes', 'Office': 'Yes', 'Professional space': 'Yes', 'Basement surface': '148 m²\nsquare meters', 'Attic surface': '36 m²\nsquare meters', 'Furnished': 'No', 'Armored door': 'No', 'Surface of the plot': '1000 m²\nsquare meters', 'Land is facing street': 'No', 'Wooded land': 'No', 'Plot at rear': 'No', 'Flat land': 'Yes', 'W

KeyboardInterrupt: 

In [1]:
from functools import lru_cache
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import time
from multiprocessing import Pool
import itertools
import json
import re
import httpx
from bs4 import BeautifulSoup
import requests

def set_up_threads(all_search_pages):
    with Pool() as pool:
        return pool.map(get_urls, all_search_pages, timeout = 60)

def get_urls(url, max):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(2)
    #cookie_button = driver.find_element(By.XPATH, "//*[@id='uc-btn-accept-banner']")
    time.sleep(1)
    #cookie_button.click()
    list_of_urls = []
    for i in range(max):
        try:
            paragraphs_with_link = driver.find_elements(By.XPATH, "//h2/a[@href]")
            for paragraph_with_link in paragraphs_with_link:
                    list_of_urls.append(paragraph_with_link.get_attribute('href'))
            #wait = WebDriverWait(driver, 10)
            #lol = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@class='pagination__link pagination__link--next button button--text button--size-small']")))
            driver.find_element(By.XPATH, "//*[@class='pagination__link pagination__link--next button button--text button--size-small']").click()
            #print(lol.get_attribute('href'))
        except StaleElementReferenceException:
            pass
        i+=1
    driver.close()
    return list_of_urls

def get_max_pages(url):
    driver = webdriver.Chrome()
    driver.get(url)
    last_page = driver.find_element(By.XPATH, "//a[@class='pagination__skip']/following::*").text
    last_page = re.sub("^Page.*\\n", "", last_page)
    driver.close()
    return int(last_page)

def save(element, name):
    with open(f"{name}.json", "w") as write_file:
        json.dump(element, write_file) 

#####################
list_of_list_of_all_urls = []
all_urls_to_cure = []
static_all_urls = []
all_data = []
url_house = 'https://www.immoweb.be/en/search/house/for-sale?countries=BE&page=1&orderBy=relevance'
url_app = 'https://www.immoweb.be/en/search/apartment/for-sale?countries=BE&page=1&orderBy=relevance'
ignored_exceptions = (NoSuchElementException,StaleElementReferenceException)
get_urls_parameters_for_pool = [(url_house,get_max_pages(url_house)),(url_app,get_max_pages(url_app))]
#get_urls_parameters_for_pool = [(url_house,2),(url_app,2)]


with Pool() as pool:
    all_urls_to_cure = list(pool.starmap(get_urls, get_urls_parameters_for_pool))
all_urls_to_cure = list(itertools.chain.from_iterable(all_urls_to_cure)) # Flat la liste de liste en liste
print(len(all_urls_to_cure))
all_urls = list(dict.fromkeys(all_urls_to_cure)) #remove duplicates 
print(len(all_urls_to_cure))
all_urls = [item for item in all_urls_to_cure if "project" not in item]
print(len(all_urls))
save(all_urls,'all_urls')

# for url in all_urls:
#     static_url = re.sub("\?.*",'',url)
#     static_all_urls.append(static_url)

# #print(static_all_urls)
# save(all_urls, 'all_urls')
# save(static_all_urls, 'static_all_urls')

Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.10/3.10.6_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python@3.10/3.10.6_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python@3.10/3.10.6_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/local/Cellar/python@3.10/3.10.6_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'get_urls' on <module '__main__' (built-in)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.10/3.10.6_2/Frameworks/Python.framewor

KeyboardInterrupt: 

In [2]:
import asyncio
import httpx
from bs4 import BeautifulSoup
import re
from requests_html import HTMLSession
import requests
url = "https://www.immoweb.be/en/classified/apartment/for-sale/deinze/9800/10130388?searchId=63341f64bc8e6"


r = requests.get(url)

soup = BeautifulSoup(r.content,"html.parser")
data = {}
for l in soup.html.find_all("tr"):
    if l.find("th"):
        a = str(l.find("th"))
        a = re.sub("<([^<]*)>", "", a)
        a = a.strip()
        a = a.replace("\n", "")
        a = a.replace("\t", "")
        if a is not None:
            th = a
        
        
    if l.find("td"):
        a = str(l.find("td"))
        a = re.sub("<([^<]*)>", "", a)
        a = a.strip()
        a = a.replace("\n", "")
        a = a.replace("\t", "")
        a = re.sub("\ {2,}",' ',a)
        td = a

    data[th] = td    

print (data)


{'Available as of': 'At delivery', 'Available date': 'February 1 2024 - 12:00 AM', 'Neighbourhood or locality': 'Gent en omstreken', 'Construction year': '2022', 'Floor': '5', 'Number of floors': '7', 'Building condition': 'As new', 'Number of frontages': '2', 'Outdoor parking spaces': '1', 'Surroundings type': 'Urban', 'Living area': '91 m² square meters', 'Living room surface': '22 m² square meters', 'Kitchen type': 'Hyper equipped', 'Kitchen surface': '4 m² square meters', 'Bedrooms': '2', 'Bedroom 1 surface': '16 m² square meters', 'Bedroom 2 surface': '10 m² square meters', 'Bathrooms': '1', 'Toilets': '1', 'Basement': 'Yes', 'Furnished': 'No', 'Terrace surface': '12 m² square meters', 'Elevator': 'Yes', 'Accessible for disabled people': 'No', 'TV cable': 'Yes', 'Visio phone': 'Yes', 'Swimming pool': 'No', 'Primary energy consumption': '1 kWh/m² kilowatt hour per square meters', 'Energy class': 'A', '': 'A', 'Reference number of the EPC report': 'Not specified', 'CO₂ emission': 'N